In [1]:
import pandas as pd
import numpy as np

from pandas.tseries.holiday import USFederalHolidayCalendar

In [2]:
df = pd.read_parquet('../data/data.parquet')

In [3]:
df.head()

,LocationAbbr,LocationDesc,Activitylimitationduetoarthritisamongadultsagedge18yearswhohavedoctor-diagnosedarthritis_Age-adjustedPrevalence,Activitylimitationduetoarthritisamongadultsagedge18yearswhohavedoctor-diagnosedarthritis_CrudePrevalence,Adultsagedge18yearswitharthritiswhohavetakenaclasstolearnhowtomanagearthritissymptoms_Age-adjustedPrevalence,Adultsagedge18yearswitharthritiswhohavetakenaclasstolearnhowtomanagearthritissymptoms_CrudePrevalence,Adultswithdiagnoseddiabetesagedge18yearswhohavetakenadiabetesself-managementcourse_Age-adjustedPrevalence,Adultswithdiagnoseddiabetesagedge18yearswhohavetakenadiabetesself-managementcourse_CrudePrevalence,Alcoholuseamongyouth_CrudePrevalence,Alcoholusebeforepregnancy_CrudePrevalence,...,Average Unemployment among people 16 years and older in the labor force,Weighted Average Crowding among housing units,Weighted Average Housing cost burden among households,Weighted Average No broadband internet subscription among households,Weighted Average No high school diploma among adults aged 25 years or older,Weighted Average Persons aged 65 years or older,Weighted Average Persons living below 150% of the poverty level,Weighted Average Persons of racial or ethnic minority status,Weighted Average Single-parent households,Weighted Average Unemployment among people 16 years and older in the labor force
0,OR,Oregon,49.9,44.6,23.2,20.2,NaN,NaN,NaN,61.8,...,6.229064,3.436493,31.307246,9.976415,8.714223,16.167922,21.08827,28.02918,5.812924,5.720091
1,OR,Oregon,49.9,44.6,23.2,20.2,NaN,NaN,NaN,61.8,...,6.229064,3.436493,31.307246,9.976415,8.714223,16.167922,21.08827,28.02918,5.812924,5.720091
2,OR,Oregon,49.9,44.6,23.2,20.2,NaN,NaN,NaN,61.8,...,6.229064,3.436493,31.307246,9.976415,8.714223,16.167922,21.08827,28.02918,5.812924,5.720091
3,OR,Oregon,49.9,44.6,23.2,20.2,NaN,NaN,NaN,61.8,...,6.229064,3.436493,31.307246,9.976415,8.714223,16.167922,21.08827,28.02918,5.812924,5.720091
4,OR,Oregon,49.9,44.6,23.2,20.2,NaN,NaN,NaN,61.8,...,6.229064,3.436493,31.307246,9.976415,8.714223,16.167922,21.08827,28.02918,5.812924,5.720091


## Look at Which Non-Numeric Columns are Needed for Modeling

In [4]:
non_numeric_columns = df.select_dtypes(exclude='number').columns.tolist()

print("Non-numeric columns:", non_numeric_columns)

Non-numeric columns: ['LocationAbbr', 'LocationDesc', 'DIVISION', 'STNAME', 'CTYNAME', 'REGION_NAME', 'week_end', 'geography', 'county', 'ed_trends_covid', 'ed_trends_influenza', 'ed_trends_rsv', 'hsa', 'hsa_counties', 'hsa_nci_id', 'trend_source', 'BuildNumber']


In [5]:
non_numeric_df = df.select_dtypes(exclude='number')
non_numeric_df.head()

,LocationAbbr,LocationDesc,DIVISION,STNAME,CTYNAME,REGION_NAME,week_end,geography,county,ed_trends_covid,ed_trends_influenza,ed_trends_rsv,hsa,hsa_counties,hsa_nci_id,trend_source,BuildNumber
0,OR,Oregon,Pacific,Oregon,Oregon,West,2022-10-08,Oregon,All,No Change,Increasing,Increasing,All,All,All,State,2025-01-03
1,OR,Oregon,Pacific,Oregon,Oregon,West,2022-11-26,Oregon,All,Increasing,Increasing,Increasing,All,All,All,State,2025-01-03
2,OR,Oregon,Pacific,Oregon,Oregon,West,2022-12-10,Oregon,All,Increasing,Increasing,Increasing,All,All,All,State,2025-01-03
3,OR,Oregon,Pacific,Oregon,Oregon,West,2022-12-24,Oregon,All,Decreasing,Decreasing,Decreasing,All,All,All,State,2025-01-03
4,OR,Oregon,Pacific,Oregon,Oregon,West,2023-01-07,Oregon,All,Decreasing,Decreasing,Decreasing,All,All,All,State,2025-01-03


In [6]:
cols_to_drop = ["LocationAbbr", 
                "LocationDesc",
                "STNAME",
                "CTYNAME",
                "REGION_NAME",
                "geography",
                "county",
                "hsa",
                "hsa_counties",
                "hsa_nci_id",
                "trend_source",
                "BuildNumber"
                ]

## Null Columns
Many states did not report chronic health. We will drop columns with less than 80% non-null values.

In [7]:
missing_percent = df.isnull().mean() * 100
print(missing_percent.describe())

count    448.000000
mean       3.815577
std       11.217997
min        0.000000
25%        0.000000
50%        0.000000
75%        0.044145
max       48.131299
dtype: float64


In [8]:
threshold = 0.8
df = df.loc[:, df.notnull().mean() >= threshold]

In [9]:
df.head()

,LocationAbbr,LocationDesc,Activitylimitationduetoarthritisamongadultsagedge18yearswhohavedoctor-diagnosedarthritis_Age-adjustedPrevalence,Activitylimitationduetoarthritisamongadultsagedge18yearswhohavedoctor-diagnosedarthritis_CrudePrevalence,Adultsagedge18yearswitharthritiswhohavetakenaclasstolearnhowtomanagearthritissymptoms_Age-adjustedPrevalence,Adultsagedge18yearswitharthritiswhohavetakenaclasstolearnhowtomanagearthritissymptoms_CrudePrevalence,Alcoholuseamongyouth_CrudePrevalence,Allteethlostamongadultsagedge65years_Age-adjustedPrevalence,Allteethlostamongadultsagedge65years_CrudePrevalence,Arthritisamongadultsagedge18years_Age-adjustedPrevalence,...,Average Unemployment among people 16 years and older in the labor force,Weighted Average Crowding among housing units,Weighted Average Housing cost burden among households,Weighted Average No broadband internet subscription among households,Weighted Average No high school diploma among adults aged 25 years or older,Weighted Average Persons aged 65 years or older,Weighted Average Persons living below 150% of the poverty level,Weighted Average Persons of racial or ethnic minority status,Weighted Average Single-parent households,Weighted Average Unemployment among people 16 years and older in the labor force
0,OR,Oregon,49.9,44.6,23.2,20.2,NaN,11.8,11.2,22.3,...,6.229064,3.436493,31.307246,9.976415,8.714223,16.167922,21.08827,28.02918,5.812924,5.720091
1,OR,Oregon,49.9,44.6,23.2,20.2,NaN,11.8,11.2,22.3,...,6.229064,3.436493,31.307246,9.976415,8.714223,16.167922,21.08827,28.02918,5.812924,5.720091
2,OR,Oregon,49.9,44.6,23.2,20.2,NaN,11.8,11.2,22.3,...,6.229064,3.436493,31.307246,9.976415,8.714223,16.167922,21.08827,28.02918,5.812924,5.720091
3,OR,Oregon,49.9,44.6,23.2,20.2,NaN,11.8,11.2,22.3,...,6.229064,3.436493,31.307246,9.976415,8.714223,16.167922,21.08827,28.02918,5.812924,5.720091
4,OR,Oregon,49.9,44.6,23.2,20.2,NaN,11.8,11.2,22.3,...,6.229064,3.436493,31.307246,9.976415,8.714223,16.167922,21.08827,28.02918,5.812924,5.720091


# Feature Engineer: Date Columns

In [10]:
df['WeekNumber'] = pd.to_datetime(df['week_end']).dt.isocalendar().week

In [11]:
def get_season(date):
    month = date.month
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

df['Season'] = pd.to_datetime(df['week_end']).apply(get_season)

## Sin and Cosine Transform Week Number and Month Columns

In [16]:
# Sine and Cosine transformations for WeekNumber
df['WeekNumber_sin'] = np.sin(2 * np.pi * df['WeekNumber'] / 52)
df['WeekNumber_cos'] = np.cos(2 * np.pi * df['WeekNumber'] / 52)

# Sine and Cosine transformations for month
df['Month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['Month_cos'] = np.cos(2 * np.pi * df['month'] / 12)

df = df.drop(columns=['WeekNumber', 'Month'])

### Holidays

In [12]:
cal = USFederalHolidayCalendar()
holidays = cal.holidays(start=df['week_end'].min(), end=df['week_end'].max())
df['IsHolidayWeek'] = pd.to_datetime(df['week_end']).isin(holidays)


## Introduce Lag Variable
I would expect current trends for infectious disease to be meaningful in predicting current trends.

In [13]:
trend_columns = ['ed_trends_influenza', 'ed_trends_rsv', 'ed_trends_covid']
for col in trend_columns:
    lag_col = f'lag_{col}'
    # Shift the values within each geography group
    df[lag_col] = df.groupby('geography')[col].shift(1, fill_value='No Change')
    # Fill any remaining NaN
    df[lag_col] = df[lag_col].fillna('No Change')

In [14]:
df.head()

,LocationAbbr,LocationDesc,Activitylimitationduetoarthritisamongadultsagedge18yearswhohavedoctor-diagnosedarthritis_Age-adjustedPrevalence,Activitylimitationduetoarthritisamongadultsagedge18yearswhohavedoctor-diagnosedarthritis_CrudePrevalence,Adultsagedge18yearswitharthritiswhohavetakenaclasstolearnhowtomanagearthritissymptoms_Age-adjustedPrevalence,Adultsagedge18yearswitharthritiswhohavetakenaclasstolearnhowtomanagearthritissymptoms_CrudePrevalence,Alcoholuseamongyouth_CrudePrevalence,Allteethlostamongadultsagedge65years_Age-adjustedPrevalence,Allteethlostamongadultsagedge65years_CrudePrevalence,Arthritisamongadultsagedge18years_Age-adjustedPrevalence,...,Weighted Average Persons living below 150% of the poverty level,Weighted Average Persons of racial or ethnic minority status,Weighted Average Single-parent households,Weighted Average Unemployment among people 16 years and older in the labor force,WeekNumber,Season,IsHolidayWeek,lag_ed_trends_influenza,lag_ed_trends_rsv,lag_ed_trends_covid
0,OR,Oregon,49.9,44.6,23.2,20.2,NaN,11.8,11.2,22.3,...,21.08827,28.02918,5.812924,5.720091,40,Fall,False,No Change,No Change,No Change
1,OR,Oregon,49.9,44.6,23.2,20.2,NaN,11.8,11.2,22.3,...,21.08827,28.02918,5.812924,5.720091,47,Fall,False,Increasing,Increasing,No Change
2,OR,Oregon,49.9,44.6,23.2,20.2,NaN,11.8,11.2,22.3,...,21.08827,28.02918,5.812924,5.720091,49,Winter,False,Increasing,Increasing,Increasing
3,OR,Oregon,49.9,44.6,23.2,20.2,NaN,11.8,11.2,22.3,...,21.08827,28.02918,5.812924,5.720091,51,Winter,False,Increasing,Increasing,Increasing
4,OR,Oregon,49.9,44.6,23.2,20.2,NaN,11.8,11.2,22.3,...,21.08827,28.02918,5.812924,5.720091,1,Winter,False,Decreasing,Decreasing,Decreasing


In [20]:
null_counts = df.isnull().sum()

# Display the result
null_counts

LocationAbbr                                                                                                         0
LocationDesc                                                                                                         0
Activitylimitationduetoarthritisamongadultsagedge18yearswhohavedoctor-diagnosedarthritis_Age-adjustedPrevalence    118
Activitylimitationduetoarthritisamongadultsagedge18yearswhohavedoctor-diagnosedarthritis_CrudePrevalence           118
Adultsagedge18yearswitharthritiswhohavetakenaclasstolearnhowtomanagearthritissymptoms_Age-adjustedPrevalence       118
                                                                                                                  ... 
lag_ed_trends_covid                                                                                                  0
WeekNumber_sin                                                                                                       0
WeekNumber_cos                                  

In [21]:
df = df.drop(columns=cols_to_drop, errors='ignore')

df.head()

,Activitylimitationduetoarthritisamongadultsagedge18yearswhohavedoctor-diagnosedarthritis_Age-adjustedPrevalence,Activitylimitationduetoarthritisamongadultsagedge18yearswhohavedoctor-diagnosedarthritis_CrudePrevalence,Adultsagedge18yearswitharthritiswhohavetakenaclasstolearnhowtomanagearthritissymptoms_Age-adjustedPrevalence,Adultsagedge18yearswitharthritiswhohavetakenaclasstolearnhowtomanagearthritissymptoms_CrudePrevalence,Alcoholuseamongyouth_CrudePrevalence,Allteethlostamongadultsagedge65years_Age-adjustedPrevalence,Allteethlostamongadultsagedge65years_CrudePrevalence,Arthritisamongadultsagedge18years_Age-adjustedPrevalence,Arthritisamongadultsagedge18years_CrudePrevalence,Arthritisamongadultsagedge18yearswhoareobese_Age-adjustedPrevalence,...,WeekNumber,Season,IsHolidayWeek,lag_ed_trends_influenza,lag_ed_trends_rsv,lag_ed_trends_covid,WeekNumber_sin,WeekNumber_cos,Month_sin,Month_cos
0,49.9,44.6,23.2,20.2,NaN,11.8,11.2,22.3,25.4,29.5,...,40,Fall,False,No Change,No Change,No Change,-0.992709,0.120537,-8.660254e-01,0.500000
1,49.9,44.6,23.2,20.2,NaN,11.8,11.2,22.3,25.4,29.5,...,47,Fall,False,Increasing,Increasing,No Change,-0.568065,0.822984,-5.000000e-01,0.866025
2,49.9,44.6,23.2,20.2,NaN,11.8,11.2,22.3,25.4,29.5,...,49,Winter,False,Increasing,Increasing,Increasing,-0.354605,0.935016,-2.449294e-16,1.000000
3,49.9,44.6,23.2,20.2,NaN,11.8,11.2,22.3,25.4,29.5,...,51,Winter,False,Increasing,Increasing,Increasing,-0.120537,0.992709,-2.449294e-16,1.000000
4,49.9,44.6,23.2,20.2,NaN,11.8,11.2,22.3,25.4,29.5,...,1,Winter,False,Decreasing,Decreasing,Decreasing,0.120537,0.992709,5.000000e-01,0.866025


In [22]:
target_columns = ['ed_trends_covid', 'ed_trends_influenza', 'ed_trends_rsv']

# Dictionary to hold the separate DataFrames
dataframes = {}

for target in target_columns:
    # Create a copy of the original DataFrame to avoid modifying it
    df_copy = df.copy()
    
    # Create the binary outcome column
    outcome_col = f"{target}_outcome"
    df_copy[outcome_col] = (df_copy[target] == 'Increasing').astype(int)
    
    # Drop the original target column
    df_copy = df_copy.drop(columns=[target])
    
    # Store the resulting DataFrame
    dataframes[target] = df_copy

covid_trends = dataframes['ed_trends_covid']
influenza_trends = dataframes['ed_trends_influenza']
rsv_trends = dataframes['ed_trends_rsv']

   Activitylimitationduetoarthritisamongadultsagedge18yearswhohavedoctor-diagnosedarthritis_Age-adjustedPrevalence  \
0                                               49.9                                                                 
1                                               49.9                                                                 
2                                               49.9                                                                 
3                                               49.9                                                                 
4                                               49.9                                                                 

   Activitylimitationduetoarthritisamongadultsagedge18yearswhohavedoctor-diagnosedarthritis_CrudePrevalence  \
0                                               44.6                                                          
1                                               44.6                 

In [25]:
influenza_trends.head()

,Activitylimitationduetoarthritisamongadultsagedge18yearswhohavedoctor-diagnosedarthritis_Age-adjustedPrevalence,Activitylimitationduetoarthritisamongadultsagedge18yearswhohavedoctor-diagnosedarthritis_CrudePrevalence,Adultsagedge18yearswitharthritiswhohavetakenaclasstolearnhowtomanagearthritissymptoms_Age-adjustedPrevalence,Adultsagedge18yearswitharthritiswhohavetakenaclasstolearnhowtomanagearthritissymptoms_CrudePrevalence,Alcoholuseamongyouth_CrudePrevalence,Allteethlostamongadultsagedge65years_Age-adjustedPrevalence,Allteethlostamongadultsagedge65years_CrudePrevalence,Arthritisamongadultsagedge18years_Age-adjustedPrevalence,Arthritisamongadultsagedge18years_CrudePrevalence,Arthritisamongadultsagedge18yearswhoareobese_Age-adjustedPrevalence,...,Season,IsHolidayWeek,lag_ed_trends_influenza,lag_ed_trends_rsv,lag_ed_trends_covid,WeekNumber_sin,WeekNumber_cos,Month_sin,Month_cos,ed_trends_influenza_outcome
0,49.9,44.6,23.2,20.2,NaN,11.8,11.2,22.3,25.4,29.5,...,Fall,False,No Change,No Change,No Change,-0.992709,0.120537,-8.660254e-01,0.500000,1
1,49.9,44.6,23.2,20.2,NaN,11.8,11.2,22.3,25.4,29.5,...,Fall,False,Increasing,Increasing,No Change,-0.568065,0.822984,-5.000000e-01,0.866025,1
2,49.9,44.6,23.2,20.2,NaN,11.8,11.2,22.3,25.4,29.5,...,Winter,False,Increasing,Increasing,Increasing,-0.354605,0.935016,-2.449294e-16,1.000000,1
3,49.9,44.6,23.2,20.2,NaN,11.8,11.2,22.3,25.4,29.5,...,Winter,False,Increasing,Increasing,Increasing,-0.120537,0.992709,-2.449294e-16,1.000000,0
4,49.9,44.6,23.2,20.2,NaN,11.8,11.2,22.3,25.4,29.5,...,Winter,False,Decreasing,Decreasing,Decreasing,0.120537,0.992709,5.000000e-01,0.866025,0


## Ordinal Encode Remaining Infectious Disease Cols

### COVID

In [ ]:
trend_mapping = {'Decreasing': 0, 'No Change': 1, 'Increasing': 2}

covid_ordinal_cols = ['ed_trends_influenza', 'ed_trends_rsv', 
                      'lag_ed_trends_influenza', 'lag_ed_trends_rsv', 'lag_ed_trends_covid']

covid_trends[covid_ordinal_cols] = covid_trends[covid_ordinal_cols].apply(lambda col: col.map(trend_mapping))

# Verify encoding
print(covid_trends[covid_ordinal_cols].head())

### Flu

In [ ]:
influenza_ordinal_cols = ['ed_trends_covid', 'ed_trends_rsv', 
                          'lag_ed_trends_influenza', 'lag_ed_trends_rsv', 'lag_ed_trends_covid']

influenza_trends[influenza_ordinal_cols] = influenza_trends[influenza_ordinal_cols].apply(lambda col: col.map(trend_mapping))

# Verify encoding
print(influenza_trends[influenza_ordinal_cols].head())


In [ ]:
rsv_ordinal_cols = ['ed_trends_covid', 'ed_trends_influenza', 
                    'lag_ed_trends_influenza', 'lag_ed_trends_rsv', 'lag_ed_trends_covid']

rsv_trends[rsv_ordinal_cols] = rsv_trends[rsv_ordinal_cols].apply(lambda col: col.map(trend_mapping))

# Verify encoding
print(rsv_trends[rsv_ordinal_cols].head())

In [24]:
covid_trends.to_parquet('../data/covid_modeling.parquet')
influenza_trends.to_parquet('../data/flu_modeling.parquet')
rsv_trends.to_parquet('../data/rsv_modeling.parquet')
